# LDA models different decades (speeches as input)

In [29]:
#System
import os
import sys
from tqdm import tqdm 

#Data structure manipulation
import pandas as pd
import numpy as np

#text cleaning 
import re
import string

#nlp
import nltk
from nltk.corpus import stopwords
import spacy

#gensim
import gensim
from gensim import corpora
from gensim.test.utils import datapath
# unlist nested lists
from itertools import chain

# count word frequencies
from collections import defaultdict

# vizualization
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

# timer function
import time

## Reading the data

In [2]:
#Path to where the raw text files are stored within the session folders, i.e, converted sessions.
origin_path = "C:/DATA/convSes"

In [3]:
#A function for reading in the speeches
def read_text(file_path, file):
    
    '''Reading the text files'''
    
    with open(file_path, 'r', encoding='utf-8') as file:
        doc=file.read()
    return doc

In [4]:
#Bringing in all the speeches
doc_set = []
for i in range (0,50):
    year = 1970 + i
    session = "session " + str(25+i)+ " - "  + str(year)
    data_path = f"{origin_path}\\{session}"
    os.chdir(data_path)
    for file in os.listdir():
        if file[-4:]=='.txt':
            file_path = f"{data_path}\\{file}"
            doc_set.append({'Year': year, 'ISO_Code': file[:3] , 'text': read_text(file_path, file)})
        else:
            print(file)
            pass

word2vec.model


In [5]:
columns=['Year', 'ISO_Code', 'text']
dataset = pd.concat([pd.DataFrame([i], columns=columns) for i in tqdm(doc_set)], ignore_index=True)
dataset.head()

100%|██████████| 8288/8288 [00:04<00:00, 1946.49it/s]


,Year,ISO_Code,text
0,1970,ALB,33: May I first convey to our President the co...
1,1970,ARG,177.\t : It is a fortunate coincidence that pr...
2,1970,AUS,100.\t It is a pleasure for me to extend to y...
3,1970,AUT,155.\t May I begin by expressing to Ambassado...
4,1970,BEL,"176. No doubt each of us, before coming up to ..."


**Creating subset including only the G20 states**

In [6]:
# only select states belonging to the G20 group (minus south afrika and EU representatives)
g20 =  dataset.ISO_Code.isin(['CAN','FRA', 'DEU', 'USA', 'GBR', 'ITA', 'JPN','ARG', 'Aus', 'BRA', 'IND', 'IDN', 
                        'CAN', 'MEX', 'RUS', 'SAU', 'KOR', 'TUR', 'CHN'])
G20 = dataset[g20]

# reset ascending index for subset dataset
G20.reset_index(inplace = True, drop = True)
G20.head()

,Year,ISO_Code,text
0,1970,ARG,177.\t : It is a fortunate coincidence that pr...
1,1970,BRA,"1.\tMr. President, I should like, first of all..."
2,1970,CAN,The General Assembly is fortunate indeed to ha...
3,1970,FRA,"84.\t Within one month, when we celebrate the..."
4,1970,GBR,"110.\t Mr. President, I should like first to s..."


In [7]:
# testing a random sample of speeches
G20.sample(10)

,Year,ISO_Code,text
629,2008,USA,I am pleased to be here to \naddress the Gener...
404,1995,IDN,On behalf of the Indonesian\ndelegation I shou...
417,1996,DEU,﻿Last year we\ntook stock of half a century of...
132,1978,JPN,﻿\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
564,2004,USA,Thank you for the honour of\naddressing the Ge...
679,2012,BRA,"﻿Once again, a woman’s\nvoice is opening the g..."
512,2001,TUR,﻿This meeting is being held at\na most difficu...
238,1985,FRA,I should like first to congratulate the Presid...
721,2014,JPN,"Humankind faces serious, \nunprecedented crise..."
135,1978,TUR,"﻿\n1.\tMr. President, on behalf of my Governme..."


**Creating subsets for the 70s,80s,90s,2000s,2010s**

In [8]:
G20_70s = G20[(G20['Year'] >= 1970) & (G20['Year'] < 1980)] 
G20_80s = G20[(G20['Year'] >= 1980) & (G20['Year'] < 1990)]
G20_90s = G20[(G20['Year'] >= 1990) & (G20['Year'] < 2000)]
G20_2000s = G20[(G20['Year'] >= 2000) & (G20['Year'] < 2010)]
G20_2010s = G20[(G20['Year'] >= 2010) & (G20['Year'] < 2020)]

## Pre-processing

**Defining/loading in the needed functions --> can we do this in a cleaner way??**

In [11]:
nlp = spacy.load('en_core_web_lg')#run in conda to download the library --> python -m download en_core_web_lg 

In [12]:
def init_proc(text, stop_words=[]):
    
    '''Pre-processing the input in single list'''
    
    stops = stopwords.words("english")
    stops.extend(stop_words)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\-', ' ', text)
    text = re.sub(r'\s\s+', ' ', text)
    text = re.sub(r'[0-9]+','', text)
    text = re.sub(r'[^\w\s]', '', text)
    lower = text.lower()
    doc = nlp(lower)
    words = []
    for token in doc:
        lemma = token.lemma_
        if lemma not in stops:
                words.append(lemma)
    return words

In [13]:
stop_words =['general', 'assembly', 'conference', 'session', 'congratulations', 
             'congratulate', 'secretarygeneral','members', 'member', 'united', 'nations', 
             'nation', 'statement', 'honour','every', 'sir', 'majesty', 'president', 
             'minister', 'prime', 'ambassador', 'thank', 'thanks', 'world', 'international', 
             'states', 'we', 'us', 'they', 'system', 'organization','say', 'think', 'know', 
             'want', 'need', 'let', 'ask', 'go', 'look', 'stand', 'open', 'give', 'see', 'come', 
             'make', 'made', 'meet','act', 'use', 'take', 'bring', 'ensure', 'able', 'assume', 
             'continue', 'change', 'progress', 'process', 'year', 'years', 'time', 'today',  
             'would', 'will', 'might', 'together', 'common', 'future', 'one', 'order', 'end', 
             'new', 'necessary', 'major', 'minor', 'many', 'people', 'peoples', 'appropriate', 
             'historic', 'adequate', 'best', 'better', 'confident', 'important', 'special',
             'great', 'therefore', 'thus', 'hence', 'like', 'particularly', 'many', 'much', 
             'greater', 'especially', 'towards', 'always', 'whether', 'around',
             'possible', 'clear', 'simply', 'must', 'also', 'however', 'mr',
             'united', 'kingdom', 'great', 'britain', 'france', 'germany','italy', 'japan',
             'canada', 'usa', 'argentina', 'australia', 'china', 'brazil', 'india', 'indonesia',  
             'mexico', 'russia', 'saudi', 'arabia', 'south', 'korea', 'turkey','liechtenstein',
             'I', ' ', '  ']

In [18]:
#loading the bigram models from the bigram_trigram_model_notebooks
#temp_file1 = datapath("C:/DATA/DTM/phrasers/bigram_speeches")
#temp_file2 = datapath("C:/DATA/DTM/phrasers/trigram_speeches")
from gensim.models.phrases import Phrases
bigram_phraser = Phrases.load("C:/DATA/DTM/phrasers/bigram_speeches")
trigam_phraser = Phrases.load("C:/DATA/DTM/phrasers/trigram_speeches")

In [20]:
#Function to preprocess speeches and also detect bi and trigrams
def pre_proc_comb(corpus,stop_words=[]):
    
    '''Looping the pre-processing over a list. Also checks if input is correct'''
    
    l = []
    if isinstance(corpus, str):
        l.append(init_proc(corpus,stop_words))
    elif all(isinstance(s, str) for s in corpus):    
        for item in tqdm(corpus):
            l.append(init_proc(item,stop_words))
    else:
        print("Error: This function only accepts strings or a list of strings.")
    bigram_token = []
    for j in l:
        bigram_token.append(bigram_phraser[j])
    trigram_token = []
    for i in bigram_token:
        trigram_token.append(trigam_phraser[i])
    return trigram_token

**Preprocessing the 5 dataslices**

In [23]:
#70s
text_corpus_70s = G20_70s['text'].values.tolist()
processed_corpus_70s = pre_proc_comb(text_corpus_70s,stop_words)

100%|██████████| 153/153 [01:51<00:00,  1.37it/s]


In [24]:
#80s
text_corpus_80s = G20_80s['text'].values.tolist()
processed_corpus_80s = pre_proc_comb(text_corpus_80s,stop_words)

100%|██████████| 160/160 [02:05<00:00,  1.27it/s]


In [25]:
#90s
text_corpus_90s = G20_90s['text'].values.tolist()
processed_corpus_90s = pre_proc_comb(text_corpus_90s,stop_words)

100%|██████████| 167/167 [01:27<00:00,  1.91it/s]


In [26]:
#2000s
text_corpus_2000s = G20_2000s['text'].values.tolist()
processed_corpus_2000s = pre_proc_comb(text_corpus_2000s,stop_words)

100%|██████████| 166/166 [00:53<00:00,  3.13it/s]


In [27]:
#2010s
text_corpus_2010s = G20_2010s['text'].values.tolist()
processed_corpus_2010s = pre_proc_comb(text_corpus_2010s,stop_words)

100%|██████████| 166/166 [01:04<00:00,  2.59it/s]


## Building the LDA models for every decade sice 

### 70s

In [33]:
%%time
# Placing the processed corpus in necessary format for LDA

dictionary_70s = corpora.Dictionary(processed_corpus_70s)
doc_term_matrix_70s = [dictionary_70s.doc2bow(speech) for speech in processed_corpus_70s]

# Creating the LDA object 

LDA = gensim.models.ldamodel.LdaModel

#building the model
lda_model_70s = LDA(corpus = doc_term_matrix_70s, id2word = dictionary_70s, num_topics = 10, random_state=100,
               chunksize= 1000, passes = 1000, iterations = 500)

Wall time: 30min 11s


In [34]:
#Saving the model locally 
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/70s")
lda_model_70s.save(temp_file)

In [35]:
#Loading the model if it is already built
LDA = gensim.models.ldamodel.LdaModel
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/70s")
lda_model_70s = LDA.load(temp_file)

In [ ]:
#investigating the word distribution over the different topics
#lda_model_70s.print_topics()

In [ ]:
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis
#pyLDAvis.enable_notebook()

In [ ]:
vis70s = gensimvis.prepare(lda_model_70s,doc_term_matrix_70s,dictionary_70s,mds= 'mmds', R=30)
vis70s

### 80s

In [37]:
%%time
# Placing the processed corpus in necessary format for LDA

dictionary_80s = corpora.Dictionary(processed_corpus_80s)
doc_term_matrix_80s = [dictionary_80s.doc2bow(speech) for speech in processed_corpus_80s]

# Creating the LDA object 

LDA = gensim.models.ldamodel.LdaModel

#building the model
lda_model_80s = LDA(corpus = doc_term_matrix_80s, id2word = dictionary_80s, num_topics = 10, random_state=100,
               chunksize= 1000, passes = 1000, iterations = 500)

Wall time: 32min 45s


In [38]:
#Saving the model locally 
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/80s")
lda_model_80s.save(temp_file)

In [39]:
#Loading the model if it is already built
LDA = gensim.models.ldamodel.LdaModel
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/80s")
lda_model_80s = LDA.load(temp_file)

In [ ]:
#investigating the word distribution over the different topics
#lda_model_80s.print_topics()

In [ ]:
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis
#pyLDAvis.enable_notebook()

In [41]:
vis80s = gensimvis.prepare(lda_model_80s,doc_term_matrix_80s,dictionary_80s,mds= 'mmds', R=30)
vis80s

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
3      32.420952   8.115230       1        1  63.636673
6       0.648955  71.707100       2        1  32.701476
5      71.019684   9.971006       3        1   1.915173
2       3.785981  33.185135       4        1   1.355517
0     -42.666500  -9.222070       5        1   0.133990
4      -3.621689 -43.526516       6        1   0.093749
8      -3.632402  -4.171708       7        1   0.077597
7     -36.584400  35.804413       8        1   0.051574
9      44.391380  50.615383       9        1   0.028671
1      40.235809 -31.591665      10        1   0.005578, topic_info=               Term         Freq        Total Category  logprob  loglift
454           peace  2552.000000  2552.000000  Default  30.0000  30.0000
204          effort  1629.000000  1629.000000  Default  29.0000  29.0000
153         country  4515.000000  4515.000000  Default  28.0000  28.0000
291      government  1170.000000  1170.000000  Default  27.0000  27.0000
490         problem  1573.000000  1573.000000  Default  26.0000  26.0000
..              ...          ...          ...      ...      ...      ...
25           affect     0.001278   197.156030  Topic10  -9.4358  -2.1521
26           affirm     0.001278    41.815895  Topic10  -9.4358  -0.6014
27   aforementioned     0.001278     7.953004  Topic10  -9.4358   1.0583
28           africa     0.001278   611.336119  Topic10  -9.4358  -3.2838
29           agenda     0.001278   108.295951  Topic10  -9.4358  -1.5530

[680 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.698863   abandon
0         2  0.283913   abandon
0         4  0.021839   abandon
1         1  0.943005    abroad
1         2  0.044905    abroad
...     ...       ...       ...
4225      1  0.553555       xxx
5777      2  0.907013  yearning
5777      6  0.113377  yearning
9140      1  0.214277       yen
9140      3  0.642830       yen

[894 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 6, 3, 1, 5, 9, 8, 10, 2])

### 90s

In [42]:
%%time
# Placing the processed corpus in necessary format for LDA

dictionary_90s = corpora.Dictionary(processed_corpus_90s)
doc_term_matrix_90s = [dictionary_90s.doc2bow(speech) for speech in processed_corpus_90s]

# Creating the LDA object 

LDA = gensim.models.ldamodel.LdaModel

#building the model
lda_model_90s = LDA(corpus = doc_term_matrix_90s, id2word = dictionary_90s, num_topics = 10, random_state=100,
               chunksize= 1000, passes = 1000, iterations = 500)

Wall time: 25min 3s


In [43]:
#Saving the model locally 
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/90s")
lda_model_90s.save(temp_file)

In [44]:
#Loading the model if it is already built
LDA = gensim.models.ldamodel.LdaModel
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/90s")
lda_model_90s = LDA.load(temp_file)

In [ ]:
#investigating the word distribution over the different topics
#lda_model_90s.print_topics()

In [ ]:
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis
#pyLDAvis.enable_notebook()

In [45]:
vis90s = gensimvis.prepare(lda_model_90s,doc_term_matrix_90s,dictionary_90s,mds= 'mmds', R=30)
vis90s

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.161115  0.159204       1        1  25.286061
1      0.089153  0.152970       2        1  23.479411
5     -0.040375  0.195419       3        1  23.098036
9      0.036671  0.207860       4        1  16.595548
3      0.205276  0.098153       5        1  10.244219
2     -0.226509 -0.064226       6        1   0.802977
7     -0.030789 -0.257152       7        1   0.361208
0     -0.084979 -0.172988       8        1   0.098540
8     -0.052163 -0.162489       9        1   0.026345
4     -0.057399 -0.156751      10        1   0.007655, topic_info=                            Term         Freq        Total Category  logprob  \
985                         long   413.000000   413.000000  Default  30.0000   
576                          two   410.000000   410.000000  Default  29.0000   
161                  development  1691.000000  1691.000000  Default  28.0000   
589                          war   518.000000   518.000000  Default  27.0000   
212                       europe   379.000000   379.000000  Default  26.0000   
..                           ...          ...          ...      ...      ...   
25   agreement_tariff_trade_gatt     0.001523    15.225028  Topic10  -9.3029   
26          agricultural_product     0.001523     3.544711  Topic10  -9.3029   
27                           aim     0.001523   197.660842  Topic10  -9.3029   
28                         allow     0.001523   207.122930  Topic10  -9.3029   
29                        almost     0.001523    86.376866  Topic10  -9.3029   

     loglift  
985  30.0000  
576  29.0000  
161  28.0000  
589  27.0000  
212  26.0000  
..       ...  
25    0.2675  
26    1.7250  
27   -2.2961  
28   -2.3429  
29   -1.4683  

[757 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
2891       2  0.708481  accelerate
2891       3  0.122152  accelerate
2891       5  0.097722  accelerate
2891       6  0.024430  accelerate
2891       7  0.024430  accelerate
...      ...       ...         ...
5053       4  0.710026  yugoslavia
5053       5  0.142005  yugoslavia
10166      1  0.273360           â
10166      2  0.410039           â
10166      6  0.341700           â

[1481 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 6, 10, 4, 3, 8, 1, 9, 5])

### 2000s

In [46]:
%%time
# Placing the processed corpus in necessary format for LDA

dictionary_2000s = corpora.Dictionary(processed_corpus_2000s)
doc_term_matrix_2000s = [dictionary_2000s.doc2bow(speech) for speech in processed_corpus_2000s]

# Creating the LDA object 

LDA = gensim.models.ldamodel.LdaModel

#building the model
lda_model_2000s = LDA(corpus = doc_term_matrix_2000s, id2word = dictionary_2000s, num_topics = 10, random_state=100,
               chunksize= 1000, passes = 1000, iterations = 500)

Wall time: 13min 52s


In [47]:
#Saving the model locally 
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/2000s")
lda_model_2000s.save(temp_file)

In [48]:
#Loading the model if it is already built
LDA = gensim.models.ldamodel.LdaModel
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/2000s")
lda_model_2000s = LDA.load(temp_file)

In [49]:
#investigating the word distribution over the different topics
#lda_model_2000s.print_topics()

In [50]:
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis
#pyLDAvis.enable_notebook()

In [51]:
vis2000s = gensimvis.prepare(lda_model_2000s,doc_term_matrix_2000s,dictionary_2000s,mds= 'mmds', R=30)
vis2000s

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.075405 -0.150550       1        1  61.865717
5      0.030908 -0.233779       2        1  12.516557
8     -0.144460 -0.192973       3        1   8.682817
4     -0.034321 -0.151920       4        1   8.152083
7      0.266847 -0.067878       5        1   6.883281
9      0.112710  0.096180       6        1   1.141857
3     -0.023071  0.220721       7        1   0.309995
2     -0.144726  0.158155       8        1   0.260933
1     -0.083525  0.168482       9        1   0.135378
0     -0.055767  0.153562      10        1   0.051382, topic_info=                              Term         Freq        Total Category  \
292                         global   743.000000   743.000000  Default   
179                    development  1174.000000  1174.000000  Default   
148                        country  1652.000000  1652.000000  Default   
555                       security  1389.000000  1389.000000  Default   
441                          peace  1062.000000  1062.000000  Default   
...                            ...          ...          ...      ...   
2149  transnational_organize_crime     0.082865     7.617067  Topic10   
3223                    liberation     0.082865     6.517831  Topic10   
3744                       undergo     0.082865    11.387136  Topic10   
4918                     celebrate     0.082865     7.614541  Topic10   
1798                        master     0.082865    10.587123  Topic10   

      logprob  loglift  
292   30.0000  30.0000  
179   29.0000  29.0000  
148   28.0000  28.0000  
555   27.0000  27.0000  
441   26.0000  26.0000  
...       ...      ...  
2149  -6.7591   3.0527  
3223  -6.7591   3.2085  
3744  -6.7591   2.6506  
4918  -6.7591   3.0530  
1798  -6.7591   2.7234  

[731 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
7742      5  0.890503      abbas
6018      2  0.981068  abduction
3148      2  0.904676   abdullah
6545      6  0.530117       ably
679       1  0.585161      abuse
...     ...       ...        ...
1449      5  0.521686      young
1449      6  0.057965      young
1449      8  0.028983      young
7463      5  0.934176      youth
8769      2  0.904602         yu

[1325 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 9, 5, 8, 10, 4, 3, 2, 1])

### 2010s

In [52]:
%%time
# Placing the processed corpus in necessary format for LDA

dictionary_2010s = corpora.Dictionary(processed_corpus_2010s)
doc_term_matrix_2010s = [dictionary_2010s.doc2bow(speech) for speech in processed_corpus_2010s]

# Creating the LDA object 

LDA = gensim.models.ldamodel.LdaModel

#building the model
lda_model_2010s = LDA(corpus = doc_term_matrix_2010s, id2word = dictionary_2010s, num_topics = 10, random_state=100,
               chunksize= 1000, passes = 1000, iterations = 500)

Wall time: 18min 4s


In [53]:
#Saving the model locally 
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/2010s")
lda_model_2010s.save(temp_file)

In [54]:
#Loading the model if it is already built
LDA = gensim.models.ldamodel.LdaModel
temp_file = datapath("C:/DATA/DTM/models/LDA_speeches/2010s")
lda_model_2010s = LDA.load(temp_file)

In [55]:
#investigating the word distribution over the different topics
#lda_model_2010s.print_topics()

In [56]:
#import pyLDAvis
#import pyLDAvis.gensim_models as gensimvis
#pyLDAvis.enable_notebook()

In [57]:
vis2010s = gensimvis.prepare(lda_model_2010s,doc_term_matrix_2010s,dictionary_2010s,mds= 'mmds', R=30)
vis2010s

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.141637  0.230005       1        1  45.198889
5      0.083412  0.109513       2        1  16.065173
2     -0.056957  0.243721       3        1  15.941687
6     -0.008698  0.145510       4        1  15.440925
4      0.284804 -0.052485       5        1   2.758152
3     -0.174192  0.079926       6        1   2.475265
0     -0.066793 -0.225772       7        1   0.732962
9     -0.211285 -0.170886       8        1   0.716597
7      0.056134 -0.214163       9        1   0.522205
1     -0.048063 -0.145370      10        1   0.148145, topic_info=             Term         Freq        Total Category  logprob  loglift
685   development  1156.000000  1156.000000  Default  30.0000  30.0000
104       country  1904.000000  1904.000000  Default  29.0000  29.0000
372         peace  1210.000000  1210.000000  Default  28.0000  28.0000
657   cooperation   495.000000   495.000000  Default  27.0000  27.0000
468      security  1239.000000  1239.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
307     magnitude     0.312140    10.007892  Topic10  -6.6472   3.0471
418         quite     0.340868    28.263786  Topic10  -6.5592   2.0969
1715  information     0.339581    51.221246  Topic10  -6.5630   1.4985
4970     congress     0.291922    19.483255  Topic10  -6.7142   2.3139
423          rate     0.247154    40.800110  Topic10  -6.8807   1.4083

[784 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2320      1  0.693407   abduction
2320      9  0.231136   abduction
5         1  0.036196      accuse
5         3  0.108589      accuse
5         4  0.832517      accuse
...     ...       ...         ...
3179      1  0.157572  young_girl
3179      2  0.078786  young_girl
3179      3  0.315144  young_girl
3179      6  0.393930  young_girl
9627      5  0.751927       ysani

[1806 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 6, 3, 7, 5, 4, 1, 10, 8, 2])